In [2]:
import json
import pandas as pd
import random
from music21 import converter, note, chord, stream
from fractions import Fraction
import numpy as np
import matplotlib.pyplot as plt
import os

def load_json_data(file_path):
    """Loads the JSON data from a file."""
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        exit()
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file '{file_path}'.")
        exit()

def extract_abc_notations(data, max_files=None):
    """Extracts ABC notation from a list of dictionaries, with an optional limit on the number of files."""
    abc_list = [item.get('abc notation') for item in data if 'abc notation' in item]
    
    if max_files is not None and isinstance(max_files, int):
        abc_list = abc_list[:max_files]  # Limit the number of files to max_files

    return abc_list

def prepare_abc_notation(selected_abc):
    """Prepares the ABC notation by adding headers if missing."""
    abc_header = ''
    if not selected_abc.startswith('X:'):
        abc_header += 'X:1\n'
    if 'K:' not in selected_abc:
        abc_header += 'K:C\n'
    return abc_header + selected_abc

def parse_abc_to_score(abc_notation):
    """Parses ABC notation to a music21 score object."""
    try:
        return converter.parse(abc_notation, format='abc')
    except Exception as e:
        print(f"Error parsing ABC notation with music21: {e}")
        return None

def extract_musical_info(score):
    """Extracts musical information from the parsed score."""
    data_rows = []
    global_onset = 0.0

    if not score.parts:
        score = stream.Score([score])  # Ensure at least one part

    for part in score.parts:
        part_offset = 0.0
        for measure in part.getElementsByClass('Measure'):
            measure_number = measure.number
            for element in measure.notesAndRests:
                pitches, durations = extract_pitches_and_durations(element)
                local_onset = element.offset
                for pitch, dur in zip(pitches, durations):
                    midi_value, pitch_name = get_midi_and_pitch_name(pitch)
                    data_rows.append({
                        'Measure': measure_number,
                        'Local Onset': local_onset,
                        'Global Onset': part_offset + local_onset,
                        'Duration': dur,
                        'Pitch': pitch_name,
                        'MIDI': midi_value
                    })
            part_offset += measure.barDuration.quarterLength  # Update part offset
    return data_rows

def extract_pitches_and_durations(element):
    """Extracts pitches and durations from a musical element."""
    if isinstance(element, note.Note):
        return [element.pitch], [element.duration.quarterLength]
    elif isinstance(element, chord.Chord):
        return element.pitches, [element.duration.quarterLength] * len(element.pitches)
    elif isinstance(element, note.Rest):
        return ['Rest'], [element.duration.quarterLength]
    return [], []

def get_midi_and_pitch_name(pitch):
    """Returns MIDI value and pitch name for a pitch, or handles 'Rest'."""
    if pitch == 'Rest':
        return None, 'Rest'
    return pitch.midi, pitch.nameWithOctave

def filter_and_adjust_durations(df, filter_zero_duration=True, adjust_fractional_duration=True):
    """Filters and adjusts durations in the DataFrame."""
    df_processed = df.copy()

    if filter_zero_duration:
        df_processed = df_processed[df_processed['Duration'] > 0]

    if adjust_fractional_duration:
        # Apply the custom rounding logic
        df_processed['Duration'] = df_processed['Duration'].apply(lambda x: round(float(Fraction(x)), 3))
        df_processed['Local Onset'] = df_processed['Local Onset'].apply(lambda x: round(float(Fraction(x)), 3))
        df_processed['Global Onset'] = df_processed['Global Onset'].apply(lambda x: round(float(Fraction(x)), 3))

    return df_processed

def create_binary_matrix(df_processed, resolution_method='auto', manual_resolution=None):
    """
    Converts a dataframe with parsed symbolic music information into a binary matrix.
    """
    # Exclude NaN values (rests) from MIDI column
    df_midi = df_processed[df_processed['MIDI'].notna()]
    # Check if df_midi is empty
    if df_midi.empty:
        print("No MIDI notes found in the processed data.")
        return None, None, None, None, None

    # Step 1: Identify the range of pitches (Y-axis)
    lowest_midi = df_midi['MIDI'].min()
    highest_midi = df_midi['MIDI'].max()

    # Ensure MIDI values are integers
    lowest_midi = int(lowest_midi)
    highest_midi = int(highest_midi)

    # Step 2: Determine the resolution (X-axis)
    if resolution_method == 'auto':
        resolution = df_processed['Duration'].min()
        # Handle case where minimum duration is zero or NaN
        if resolution <= 0 or pd.isna(resolution):
            # Use the smallest non-zero duration
            resolution = df_processed[df_processed['Duration'] > 0]['Duration'].min()
        print(f"Using automatic resolution based on smallest duration: {resolution}")
    elif resolution_method == 'manual':
        if manual_resolution is None:
            raise ValueError("manual_resolution must be provided when resolution_method is 'manual'.")
        resolution = manual_resolution
        print(f"Using manual resolution: {resolution}")
    elif resolution_method == 'standard':
        resolution = 0.25
        print(f"Using standard resolution: {resolution}")
    else:
        raise ValueError("Invalid resolution_method. Choose from 'auto', 'manual', 'standard'.")

    # Validate that resolution is positive
    if resolution <= 0 or pd.isna(resolution):
        print("Invalid resolution calculated. Skipping this ABC notation.")
        return None, None, None, None, None

    # Calculate the total duration of the piece
    total_duration = df_processed['Global Onset'].max() + df_processed['Duration'].max()

    # Number of columns is total duration divided by resolution
    num_cols = int(np.ceil(total_duration / resolution))
    print(f"Total duration: {total_duration}, Number of columns: {num_cols}")

    # Number of rows is highest MIDI - lowest MIDI + 1
    num_rows = highest_midi - lowest_midi + 1
    num_rows = int(num_rows)

    # Step 3: Initialize a binary matrix
    binary_matrix = np.zeros((num_rows, num_cols), dtype=int)

    # Step 4: Fill in the binary matrix
    for index, row in df_processed.iterrows():
        # Skip rests
        if pd.isnull(row['MIDI']):
            continue

        # Calculate the row corresponding to the MIDI number (flipping it)
        row_index = highest_midi - int(row['MIDI'])

        # Calculate the columns corresponding to the Global Onset and Duration
        start_col = int(np.floor(row['Global Onset'] / resolution))
        end_col = int(np.ceil((row['Global Onset'] + row['Duration']) / resolution))

        # Handle edge cases where end_col might exceed num_cols
        end_col = min(end_col, num_cols)

        # Set the appropriate elements to 1
        binary_matrix[row_index, start_col:end_col] = 1

    return binary_matrix, lowest_midi, highest_midi, num_cols, resolution

def process_abc_notation(abc_notation, filter_zero_duration=True, adjust_fractional_duration=True, resolution_method='auto', manual_resolution=None):
    """Processes a single ABC notation string and returns the binary matrix and associated info."""
    # Prepare the ABC notation
    abc_full = prepare_abc_notation(abc_notation)

    # Parse with music21
    score = parse_abc_to_score(abc_full)
    if score is None:
        return None  # Skip if parsing failed

    # Extract musical information
    data_rows = extract_musical_info(score)
    if not data_rows:
        print("No musical information extracted from the score.")
        return None  # Skip if no data

    # Create and process DataFrame
    df_notes = pd.DataFrame(data_rows).sort_values(by=['Global Onset']).reset_index(drop=True)

    # Filter and Adjust Data
    df_processed = filter_and_adjust_durations(
        df_notes,
        filter_zero_duration=filter_zero_duration,
        adjust_fractional_duration=adjust_fractional_duration
    )
    if df_processed.empty:
        print("Processed DataFrame is empty after filtering.")
        return None  # Skip if no data

    # Create Binary Matrix
    binary_matrix, low_midi, high_midi, num_cols, resolution = create_binary_matrix(
        df_processed,
        resolution_method=resolution_method,
        manual_resolution=manual_resolution
    )
    if binary_matrix is None:
        print("Binary matrix creation failed due to empty data.")
        return None  # Skip if no data

    return {
        'binary_matrix': binary_matrix,
        'lowest_midi': low_midi,
        'highest_midi': high_midi,
        'num_cols': num_cols,
        'resolution': resolution,
        'df_processed': df_processed
    }

# Configuration flags
FILTER_ZERO_DURATION = True
ADJUST_FRACTIONAL_DURATION = True
RESOLUTION_METHOD = 'auto'  # 'auto', 'manual', or 'standard'
MANUAL_RESOLUTION = None  # Set if RESOLUTION_METHOD is 'manual'

# Step 1: Load JSON Data
json_file = 'train.json'  # Update file path as needed
data = load_json_data(json_file)

# Set the maximum number of ABC files to process
MAX_FILES = 10000  # Change this value as needed, or set to None for no limit

# Step 2: Extract ABC Notation Examples with a limit
abc_notations = extract_abc_notations(data, max_files=MAX_FILES)

# Directory to save binary matrices
output_dir = 'irish_binary_auto2'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop over each ABC notation
for idx, abc in enumerate(abc_notations):
    print(f"\nProcessing ABC notation {idx+1}/{len(abc_notations)}")

    result = process_abc_notation(
        abc_notation=abc,
        filter_zero_duration=FILTER_ZERO_DURATION,
        adjust_fractional_duration=ADJUST_FRACTIONAL_DURATION,
        resolution_method=RESOLUTION_METHOD,
        manual_resolution=MANUAL_RESOLUTION
    )

    if result is None:
        print(f"Skipping ABC notation at index {idx} due to parsing error or empty data.")
        continue

    binary_matrix = result['binary_matrix']
    lowest_midi = result['lowest_midi']
    highest_midi = result['highest_midi']
    num_cols = result['num_cols']
    resolution = result['resolution']
    df_processed = result['df_processed']

    # Optionally, save the binary matrix to a file
    matrix_filename = f'binary_matrix_{idx}.npy'
    np.save(os.path.join(output_dir, matrix_filename), binary_matrix)

    # Optionally, save the processed DataFrame to CSV
    # df_filename = f'processed_data_{idx}.csv'
    # df_processed.to_csv(os.path.join(output_dir, df_filename), index=False)

    # Optionally, plot and save the binary matrix as an image
    # plt.figure(figsize=(12, 6))
    # extent = [0, num_cols * resolution, lowest_midi, highest_midi]
    # plt.imshow(binary_matrix, aspect='auto', origin='upper', cmap='gray_r', extent=extent)
    # plt.xlabel('Time (in duration units)')
    # plt.ylabel('MIDI Number')
    # plt.title(f'Binary Matrix Representation of Piano Roll (ABC notation {idx})')
    # plt.colorbar(label='Note On/Off', ticks=[0, 1])
    # plt.tight_layout()
    # plot_filename = f'binary_matrix_{idx}.png'
    # plt.savefig(os.path.join(output_dir, plot_filename))
    # plt.close()

    print(f"Saved binary matrix and processed data for ABC notation {idx}.")

print("\nBatch processing completed.")



Processing ABC notation 1/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 44.5, Number of columns: 89
Saved binary matrix and processed data for ABC notation 0.

Processing ABC notation 2/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 75.0, Number of columns: 300
Saved binary matrix and processed data for ABC notation 1.

Processing ABC notation 3/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 36.0, Number of columns: 144
Saved binary matrix and processed data for ABC notation 2.

Processing ABC notation 4/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 54.0, Number of columns: 108
Saved binary matrix and processed data for ABC notation 3.

Processing ABC notation 5/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 48.25, Number of columns: 193
Saved binary matrix and processed data for ABC notation 4.

Processing ABC n

abcFormat: WARNING: Could not get pitch information from note:  SOSOO, assuming C


Saved binary matrix and processed data for ABC notation 79.

Processing ABC notation 81/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 59.0, Number of columns: 118
Saved binary matrix and processed data for ABC notation 80.

Processing ABC notation 82/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 34.75, Number of columns: 139
Saved binary matrix and processed data for ABC notation 81.

Processing ABC notation 83/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 64.5, Number of columns: 129
Saved binary matrix and processed data for ABC notation 82.

Processing ABC notation 84/10000
Using automatic resolution based on smallest duration: 1.0
Total duration: 98.0, Number of columns: 98
Saved binary matrix and processed data for ABC notation 83.

Processing ABC notation 85/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 58.0, Number of columns: 116
Saved bin

abcFormat: WARNING: Could not get pitch information from note:  SOSOO, assuming C


Using automatic resolution based on smallest duration: 0.5
Total duration: 54.0, Number of columns: 108
Saved binary matrix and processed data for ABC notation 281.

Processing ABC notation 283/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 56.667, Number of columns: 227
Saved binary matrix and processed data for ABC notation 282.

Processing ABC notation 284/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 47.0, Number of columns: 94
Saved binary matrix and processed data for ABC notation 283.

Processing ABC notation 285/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 77.5, Number of columns: 310
Saved binary matrix and processed data for ABC notation 284.

Processing ABC notation 286/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 68.5, Number of columns: 137
Saved binary matrix and processed data for ABC notation 285.

Processing ABC notation 287/100

...

Using automatic resolution based on smallest duration: 0.25
Total duration: 48.0, Number of columns: 192
Saved binary matrix and processed data for ABC notation 9479.

Processing ABC notation 9481/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 36.5, Number of columns: 73
Saved binary matrix and processed data for ABC notation 9480.

Processing ABC notation 9482/10000
Using automatic resolution based on smallest duration: 0.333
Total duration: 71.0, Number of columns: 214
Saved binary matrix and processed data for ABC notation 9481.

Processing ABC notation 9483/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 128.0, Number of columns: 512
Saved binary matrix and processed data for ABC notation 9482.

Processing ABC notation 9484/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 72.0, Number of columns: 288
Saved binary matrix and processed data for ABC notation 9483.

Processing ABC notat

abcFormat: WARNING: Could not get pitch information from note:  PMx9, assuming C


Using automatic resolution based on smallest duration: 0.25
Total duration: 84.0, Number of columns: 336
Saved binary matrix and processed data for ABC notation 9578.

Processing ABC notation 9580/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 72.5, Number of columns: 145
Saved binary matrix and processed data for ABC notation 9579.

Processing ABC notation 9581/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 35.5, Number of columns: 142
Saved binary matrix and processed data for ABC notation 9580.

Processing ABC notation 9582/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 51.0, Number of columns: 102
Saved binary matrix and processed data for ABC notation 9581.

Processing ABC notation 9583/10000
Using automatic resolution based on smallest duration: 0.125
Total duration: 143.5, Number of columns: 1148
Saved binary matrix and processed data for ABC notation 9582.

Processing ABC nota

abcFormat: WARNING: Could not get pitch information from note:  SOSOO, assuming C


Using automatic resolution based on smallest duration: 0.5
Total duration: 128.0, Number of columns: 256
Saved binary matrix and processed data for ABC notation 9653.

Processing ABC notation 9655/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 55.0, Number of columns: 220
Saved binary matrix and processed data for ABC notation 9654.

Processing ABC notation 9656/10000
Using automatic resolution based on smallest duration: 0.5
Total duration: 64.0, Number of columns: 128
Saved binary matrix and processed data for ABC notation 9655.

Processing ABC notation 9657/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 72.0, Number of columns: 288
Saved binary matrix and processed data for ABC notation 9656.

Processing ABC notation 9658/10000
Using automatic resolution based on smallest duration: 0.25
Total duration: 56.0, Number of columns: 224
Saved binary matrix and processed data for ABC notation 9657.

Processing ABC notati